In [1]:
import pandas as pd
df=pd.read_csv("emotion_dataset_2.csv")
df['Emotion'].unique()

array(['neutral', 'joy', 'sadness', 'fear', 'surprise', 'anger', 'shame',
       'disgust'], dtype=object)

In [5]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
data = pd.read_csv('emotion_dataset_2.csv')

# Drop rows with missing values
data.dropna(subset=['Clean_Text', 'Emotion'], inplace=True)

# Prepare the data
text = data['Clean_Text'].astype(str).values
labels = data['Emotion'].values

# Encode labels to numeric values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(text)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


Epoch 1/10
859/859 [==============================] - 139s 158ms/step - loss: 1.3765 - accuracy: 0.4995 - val_loss: 1.0860 - val_accuracy: 0.6194
Epoch 2/10
859/859 [==============================] - 131s 153ms/step - loss: 0.8489 - accuracy: 0.7109 - val_loss: 1.0541 - val_accuracy: 0.6382
Epoch 3/10
859/859 [==============================] - 135s 157ms/step - loss: 0.5799 - accuracy: 0.8076 - val_loss: 1.1091 - val_accuracy: 0.6419
Epoch 4/10
859/859 [==============================] - 135s 157ms/step - loss: 0.4317 - accuracy: 0.8586 - val_loss: 1.2479 - val_accuracy: 0.6322
Epoch 5/10
859/859 [==============================] - 128s 149ms/step - loss: 0.3454 - accuracy: 0.8860 - val_loss: 1.4062 - val_accuracy: 0.6333
Epoch 6/10
859/859 [==============================] - 135s 158ms/step - loss: 0.2942 - accuracy: 0.9026 - val_loss: 1.4685 - val_accuracy: 0.6174
Epoch 7/10
859/859 [==============================] - 158s 184ms/step - loss: 0.2546 - accuracy: 0.9163 - val_loss: 1.5400 -